# 第5章　設定ファイル

PostgreSQLには、postgresql.confとpg_hba.confという設定ファイルがある。

・PostgreSQL.conf：パラメータを設定するファイル<br>
・pg_hba.conf：クライアント認証を設定するファイル

## 5-1　postgresql.conf

・パラメータは、**1行に1つずつ設定**する<br>
・パラメータ名と設定値の間の**イコール（=）は省略**できる

指定できる値は、パラメータごとに限定されている。

In [ ]:
log_connections # 論理値
port = 5432 # 整数
cpu_tuple_cost = 0.01 # 浮動小数点
log_directory # 文字列
log_min_messages = ERROR # 列挙値
shared_buffers = 256MB # メモリの単位
log_rotation_age = 1d # 時間の単位

各パラメータの設定反映のタイミングは、以下のように分類される。

・不可（internal）：ユーザは設定変更できない<br>
・起動（postmaster）：PostgreSQLの起動／起動中：PostgreSQLの**再起動**<br>
・再読み込み（signup）：PostgreSQLの起動 or pg_ctl_reloadの再読み込み／起動中：pg_ctl_reloadのみ<br>
・スーパーユーザのみ（suset）：PostgreSQLの起動 or 再読み込み or スーパーユーザによるSET文<br>
・いつでも（user）：PostgreSQLの起動 or 再読み込み or 一般ユーザによるSET文

pg_ctl_reloadはPostgreSQL全体に設定変更が反映されるが、SET文は**実行したセッションのみ**に効果がある。

In [ ]:
# まず現在の設定値を、SHOW文で見ておく
psql -c 'SHOW max_connections'

100

In [ ]:
# postgresql.confで設定値を100から20に変更する
vi $PGDATA/postgresql.conf

In [ ]:
# pg_ctl_reloadでPostgresql.confを再読み込みする
pg_ctl reload -D $PGDATA

In [ ]:
# 設定変更が反映されていないことを確認
psql -c 'SHOW max_connections'

100

正しい方法で、パラメータを設定変更していく。

In [ ]:
# PostgreSQLを再起動する
pg_ctl restart -D $PGDATA

In [ ]:
# 設定値が20に変更されたことを確認
psql -c 'SHOW max_connections'

20

## 5-2　接続と認証

### 5-2-1　listen_addresses

「PostgreSQLが監視する**PostgreSQL側**のIPアドレス」を設定する。<br>
そうすると、PostgreSQLは、設定されたIPアドレスに届いた接続要求のみを受け付ける。

・デフォルト：localhost<br>
・設定反映のタイミング：起動

### 5-2-2　port

PostgreSQLがクライアントからの接続を受け付けるポート番号を設定する。

・デフォルト：5432<br>
・設定反映のタイミング：起動

### 5-2-3　max_connections

PostgreSQLに同時接続可能な最大数を設定する。

・デフォルト：100<br>
・設定反映のタイミング：起動

## 5-3　クライアント接続デフォルト

### 5-3-1　search_path

スキーマ検索パスを設定する。

・デフォルト："$user", public<br>
・設定反映のタイミング：いつでも

### 5-3-2　default_transaction_isolation

トランザクションのデフォルトの分離レベルを設定する。

・デフォルト：read committed<br>
・設定反映のタイミング：いつでも

### 5-3-3　client_encoding

クライアントエンコーディングを設定する。

・デフォルト：SQL_ASCII（クライアントとデータベース間の自動的な文字の変換は行われない）<br>
・設定反映のタイミング：いつでも

In [ ]:
# クライアントからのSQLをSJISとして扱い、SQLの実行結果をSJISに変換してから返す
client_encoding = 'SJIS'

## 5-4　エラー報告とログ取得

### 5-4-1　log_destination

PostgreSQLサーバログの出力先を設定する。

・デフォルト：stderr<br>
・設定反映のタイミング：再読み込み

ログの出力先には、以下の4つがある。

・stderr：サーバログを**標準エラー出力**に出力する（**平文**形式）<br>
・csvlog：サーバログを**標準エラー出力**に出力する（**CSV**形式）<br>
・syslog：サーバログを**syslog**に出力する<br>
・eventlog：サーバログを**eventlog**に出力する

### 5-4-2　logging_collector

サーバログをファイルに書き込む（リダイレクト）するかの設定を行う。<br>

・デフォルト：off<br>
・設定反映のタイミング：起動

ログは必ず残すように、onにすることが重要。

### 5-4-3　log_directory

ログファイルを格納するディレクトリを設定する。

・デフォルト：log($PGDATA/log）<br>
・設定反映のタイミング：再読み込み

絶対パス、もしくは$PGDATAディレクトリに対する相対パスで指定すること。

### 5-4-4　log_filename

ログファイルの名前を設定する。

・デフォルト：postgresql-%Y-%m-%d_%H%M%S.log（日時の情報が組み込まれる）<br>
・設定反映のタイミング：再読み込み

### 5-4-5　log_min_messages

どのレベル（ログレベル）のメッセージをサーバログに出力するかを設定する。

・デフォルト：WARNING（設定したレベル以上のものだけが出力される）<br>
・設定反映のタイミング：スーパーユーザのみ

ログレベルは、以下の5つがある。

・PANIC：致命的なエラーで、すべてのセッションが強制的に切断され、PostgreSQLは停止する<br>
・FATEL：特定のセッションで問題が起きたことを示し、そのセッションだけ切断される<br>
・LOG：データベースの管理者が着目すべきPostgreSQLの動作ログを意味する<br>
・ERROR：特定のトランザクションで問題が起きたことを示し、そのトランザクションだけがアボートされる<br>
・WARNING：想定外の動作に対する警告メッセージを意味する

### 5-4-6　log_line_prefix

サーバログの行頭に出力する内容を設定する。

・デフォルト：%m[%p]<br>
・設定反映のタイミング：再読み込み

%文字は、以下の6つがある。

・%u：データベースユーザ名<br>
・%d：データベース名<br>
・%p：プロセスID<br>
・%t：タイムスタンプ<br>
・%m：ミリ秒付きタイムスタンプ<br>
・%%：%の文字そのもの

In [ ]:
# ログメッセージの先頭に、次のように時刻とプロセスIDが出力される
log_line_prefix = '%t[%p]'

## 5-5　SET文／SHOW文の使い方

### 5-5-1　SET文

SETは、PostgreSQLの起動中にパラメータの設定値を変更することができる。<br>
ただし変更できるのは、設定反映のタイミングが「スーパーユーザのみ」もしくは「いつでも」などの**一部のパラメータ**である。

### 5-5-2　SHOW文

SHOW文は、パラメータの設定値を表示する。

In [ ]:
psql

# 現在の設定値を確認
SHOW search_path;

-----
"$user", public

# 新しいスキーマを作成
CREATE SCHEMA exam_schema;

# SET文でスキーマをsearch_pathに登録
SET search_path TO 'exam_schema';

# 設定変更が反映されていることを確認する
SHOW search_path;

-----
exam_schema

## 5-6　pg_hba.conf

クライアント認証を設定するファイルである。

・1行に1つずつ設定する<br>
・クライアント認証は、**複数フィールド**で構成され、それらは少なくとも**1つのタブもしくは空白**で区切られている必要がある<br>・フィールド内に空白を含めるときは、設定値を**ダブルクォート（''）**で囲む<br>
・設定の反映タイミングは**再読み込み**である（PostgreSQLの起動 or pg_ctl_reloadによる再読み込み）

接続方式には、以下の2つがある。

・local：**UNIXドメイン接続**に対応する（CDIRアドレス、もしくはIPアドレスとネットマスクを設定する必要がある）<br>
・host：**TCP/IP接続**に対応する

認証方式には、以下の5つがある。

・trust：接続を常に許可する<br>
・reject：接続を常に拒否する<br>
・md5：接続に対して、**MD5で暗号化されたパスワードの認証**を行う<br>
・scram-sha-256：接続に対して、**md5より堅牢なパスワードの認証**を行う<br>
・password：接続に対して、**平文のパスワードの認証**を行う

In [ ]:
# UNIXドメインを使った全ての接続を許可する（pg_hba.confのデフォルト設定）
local all all trust

# ループバックアドレス（127.0.0.1）からの全ての情報を許可する（pg_hba.confのデフォルト設定）
host all all 127.0.0.1/32 trust

# ホスト（192.168.16.8）から、データベースユーザ（user1）がデータベース（examdb）にTCP/IPで接続するのを拒否する
host examdb user1 192.168.16.8/32 reject

# ホスト192.168.16.1～192.168.16.254（サブネットアドレス192.168.16.0）上の全てのデータベースユーザについて
# データベース（examdb）に接続するときに、パスワードを認証する
# パスワード認証が成功したときだけ、接続は許可される
host examdb all 192.168.16.0/24 md5

# 上記と同じ設定だが、CIDR形式の代わりに、IPアドレスとネットマスクでサブネットアドレスを指定している
host examdb all 192.18.16.0  255.255.255.0 md5

尚、pg_hba.confでは、一致する行のうち、よりファイルの先頭にあるものが評価されるため<br>
上記では3番目の設定に従って、その接続は拒否される。

・CIDRアドレス：192.168.16.0/24<br>
・IPアドレスとネットマスク：192.18.16.0 　255.255.255.0

以下は、CIDRアドレス（接続方式がhostの場合のみ有効）の設定例である。

・192.168.100.0/24 → サブネットアドレスは192.168.100**.0** → IPアドレスは192.168.100.1～192.168.100.254が一致

CIDRアドレスには、以下のような設定方法がある。

・全てのIPアドレスに一致させたい場合：0.0.0.0/0<br>
・1つのIPアドレスのみに一致させたい場合：192.18.100.99**/32**（マスク長を**32**にする）